In [3]:
with open('../data/train_data/train.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
labels = []
sentences = []
sentence = ''
for line in lines:
    if line != '\n':
        labels.append(line[2:].strip('\n'))
        sentence += line[0]
    else:
        sentences.append(sentence)
        sentence = ''

100%|██████████| 2288790/2288790 [00:01<00:00, 1180096.38it/s]


In [4]:
with open('../data/preliminary_test_a/sample_per_line_train.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [32]:
with open('./extend_data/denoised_train_data.txt', 'r', encoding='utf-8') as f:
    lines1 = f.readlines()
# with open('./submit/labels_merged_res.txt', 'r', encoding='utf-8') as f:
#     lines2 = f.readlines()
with open('./extend_data/extend_labels_merged_res_10w.txt', 'r', encoding='utf-8') as f:
    lines3 = f.readlines()
with open('./mix_train_data_de_14w.txt', 'w', encoding='utf-8') as f:
    for line in lines1:
        f.write(line)
    # for line in lines2:
    #     f.write(line)
    for line in lines3:
        f.write(line)

In [23]:
with open('../data/train_data/unlabeled_train_data.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
import random
random.shuffle(lines)
with open('./sample_per_line_extend_data.txt', 'w', encoding='utf-8') as f:
    for line in lines[:420000]:
        f.write(line)

### 训练集去噪

In [1]:
import os
from collections import Counter


class Args:
    def __init__(self):
        self.save_path = './submit/'
        self.model_type = 'train_cv_model'
        self.fold = 10
        self.extend_save_path = './extend_data/'
        self.save_name = 'train_denoised__data.txt'


args = Args()


def denoising(args):
    save_path = os.path.join(args.save_path, args.model_type)
    all_labels = []
    chars = []
    true_label = []

    path = os.path.join(
        save_path, f'{args.model_type}-1.txt')
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        true_label = lines
        for line in lines:
            if line != '\n':
                chars.append(line[0])
            else:
                chars.append('\n')

    for fold in range(1, args.fold+1):
        labels = []
        path = os.path.join(
            save_path, f'{args.model_type}-{fold}.txt')
        with open(path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                if line != '\n':
                    labels.append(line[2:].strip('\n'))
                else:
                    labels.append(line)
        all_labels.append(labels)

    merged_label = []
    for row in zip(*all_labels):
        label = Counter(row).most_common(n=1)
        merged_label.append(label[0])

# TODO
    out_labels = []
    for d_label, t_label in zip(merged_label, true_label):
        d_l, d_n = d_label
        if t_label == '\n':
            out_labels.append('\n')
        else:
            if d_n == 10:
                char = t_label[:2]
                out_labels.append(char + d_l)
            else:
                out_labels.append(t_label.strip('\n'))

    with open(os.path.join(args.extend_save_path, args.save_name), 'w', encoding='utf-8') as f:
        for line in out_labels:
            if line == '\n':
                f.write('\n')
            else:
                f.write(line + '\n')

denoising(args)


In [3]:
with open(os.path.join(args.extend_save_path, args.save_name), 'r', encoding='utf-8') as f:
    d_lines = f.readlines()
with open('../data/train_data/train.txt', 'r', encoding='utf-8') as f:
    t_lines = f.readlines()   

with open('./extend_data/diff.txt', 'w', encoding='utf-8') as f:
    for d, t in zip(d_lines, t_lines):
        if d != t:
            f.write(t.strip('\n') + '\t' + d)